In [1]:
# Lire le dictionnaire de monsieur Bucknam
# Jupyter se base sur Pandas Tricks

In [2]:
import pandas as pd

# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dictionary/1757_Buchanan-J.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
8951,1,naif,F.,NaN,NaN,a.,8.0,145
7838,1,lĭft,NaN,NaN,NaN,v.,32.0,130
12290,3,sĕʹctăry,NaN,NaN,NaN,n.,45.0,192
11049,1,quoins,F.,NaN,NaN,n.,21.0,175
12405,1,ſhrewd,NaN,NaN,NaN,a.,16.0,194


In [3]:
cheminDuDicoBuchanan = "./../data/dictionary/Buchanan_PronChar_counts.csv" 

dfDicoBuchanan = pd.read_csv(cheminDuDicoBuchanan, delimiter=";", encoding="utf8")

dfDicoBuchanan.fillna('',inplace=True)
# afficher à l'écran un échantillon du tableau
dfDicoBuchanan.head(n=5)

,char,occ,graph
0,a,3207,a
1,ʹ,12725,
2,b,2288,b
3,ă,4533,a
4,c,5500,c


In [4]:
dfDicoBuchanan['char'].value_counts()

р    1
x    1
D    1
c    1
á    1
    ..
.    1
r    1
ẇ    1
ĕ    1
у    1
Name: char, Length: 73, dtype: int64

### 1. une colonne motVedette (transformation/simplification de la prononciation d'origine) en utilisant le mapping du fichier Buchanan_PronChar_counts.csv

In [5]:
def pron2graph(word, dfMapping = dfDicoBuchanan):
    list_of_char = []
    for char in word:
        list_of_char.append(dfMapping[dfMapping['char']==char].iloc[0]['graph'])

    return ''.join(list_of_char)

In [6]:
dfDicoBuch['motVedette'] = dfDicoBuch['pron'].apply(pron2graph)

In [7]:
dfDicoBuch.head()

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned


In [10]:
dfDicoBuch[dfDicoBuch["motVedette"].isnull()]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette


In [11]:
dfDicoBuch[dfDicoBuch["motVedette"]=='null']

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
9171,1,nŭll,L.,NaN,NaN,a.,4.0,149,null


### 2.une colonne Sampa et une colonne Ipa à partir des mots vedettes obtenus (transformation/croisement de données) en utilisant ./../data/dataframe/BigiRichSample.csv

#### BONUS : isoler les mots de Buchanan qui n'existent pas dans Bigi. Tenter de comprendre les différentes causes et tenter d'expliquer comment résoudre les différents problèmes (bonus)

In [27]:
cheminDuDicoBigiRich = "./../data/dataframe/BigiRichSample.csv" 

dfDicoBigiRich = pd.read_csv(cheminDuDicoBigiRich, delimiter=";", encoding="utf8")
dfDicoBigiRich

,hwd,pronSampa,ipa
0,a,@,ə
1,a(2),eI,?
2,a'ready,@ r\ E d i:,əɹɛdiː
3,a's,eI z,?z
4,a.,eI,?
...,...,...,...
131098,zyuganov's(3),z u g A n A v z,zugɑnɑvz
131099,zyuganov's(4),z u g A n A f s,zugɑnɑfs
131100,zywicki,z i w I k i:,ziwɪkiː
131101,zzzz,z i: z,ziːz


In [28]:
def pronSampa_and_ipa(motVedette, dfMapping = dfDicoBigiRich):
    extract = dfMapping[dfMapping['hwd'] == motVedette]
    try:
        pronSampa = extract.iloc[0]["pronSampa"]
        ipa = extract.iloc[0]["ipa"]
        nb_valeurs_trouvee = extract.shape[0]
        return (motVedette,pronSampa,ipa,nb_valeurs_trouvee)
    except:
        return(motVedette,'mot inexistant','mot inexistant',0)

In [29]:
df_transit = pd.DataFrame(dfDicoBuch['motVedette'].apply(pronSampa_and_ipa))

In [30]:
df_transit

,motVedette
0,"(abacus, { b @ k @ s, æbəkəs, 1)"
1,"(abaft, mot inexistant, mot inexistant, 0)"
2,"(abaisance, mot inexistant, mot inexistant, 0)"
3,"(abandon, @ b { n d @ n, əbændən, 1)"
4,"(abandoned, @ b { n d @ n d, əbændənd, 1)"
...,...
15795,"(zoography, mot inexistant, mot inexistant, 0)"
15796,"(zoophytes, mot inexistant, mot inexistant, 0)"
15797,"(zoophoric, mot inexistant, mot inexistant, 0)"
15798,"(zootomist, mot inexistant, mot inexistant, 0)"


In [31]:
df_pronSampa_ipa = pd.DataFrame([elem[0] for elem in df_transit.values.tolist()], columns=['motVedette','pronSampa','ipa',"nb"])
df_pronSampa_ipa

,motVedette,pronSampa,ipa,nb
0,abacus,{ b @ k @ s,æbəkəs,1
1,abaft,mot inexistant,mot inexistant,0
2,abaisance,mot inexistant,mot inexistant,0
3,abandon,@ b { n d @ n,əbændən,1
4,abandoned,@ b { n d @ n d,əbændənd,1
...,...,...,...,...
15795,zoography,mot inexistant,mot inexistant,0
15796,zoophytes,mot inexistant,mot inexistant,0
15797,zoophoric,mot inexistant,mot inexistant,0
15798,zootomist,mot inexistant,mot inexistant,0


In [32]:
df = dfDicoBuch.merge(df_pronSampa_ipa, how='left', left_on='motVedette', right_on='motVedette')
df

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,pronSampa,ipa,nb
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs,1
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,mot inexistant,mot inexistant,0
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,mot inexistant,mot inexistant,0
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən,1
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15835,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,mot inexistant,mot inexistant,0
15836,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,mot inexistant,mot inexistant,0
15837,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,mot inexistant,mot inexistant,0
15838,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,mot inexistant,mot inexistant,0


In [33]:
# 1/3 des mots n'existent pas dans le dictionnaire
df['nb'].value_counts()

1    10365
0     5475
Name: nb, dtype: int64

In [34]:
df[df['nb']==0]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,pronSampa,ipa,nb
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,mot inexistant,mot inexistant,0
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,mot inexistant,mot inexistant,0
5,2,ăbāsʹe,F.,NaN,NaN,v.,7.0,19,abase,mot inexistant,mot inexistant,0
6,3,ăbāʹſ_e_ment,F.,NaN,NaN,n.,8.0,19,abasement,mot inexistant,mot inexistant,0
10,2,ăʹbbĕſs,NaN,NaN,NaN,n.,12.0,19,abbess,mot inexistant,mot inexistant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15835,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,mot inexistant,mot inexistant,0
15836,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,mot inexistant,mot inexistant,0
15837,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,mot inexistant,mot inexistant,0
15838,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,mot inexistant,mot inexistant,0


In [35]:
df[df['pron'].str.contains("ʹ")]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,pronSampa,ipa,nb
0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs,1
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,mot inexistant,mot inexistant,0
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,mot inexistant,mot inexistant,0
3,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən,1
4,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15835,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,mot inexistant,mot inexistant,0
15836,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,mot inexistant,mot inexistant,0
15837,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,mot inexistant,mot inexistant,0
15838,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,mot inexistant,mot inexistant,0


In [36]:
df.to_csv('./../data/dataframe/BuchananRich.csv', index = False)